In [1]:
from ftplib import FTP
import os

In [2]:
def rinex(ftp: FTP):
    hours = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']

    ftp.cwd('/gnss/data/')
    rinex_type = input('Rinex type (n - nav, o - obs etc.): ')
    observation_type = input('Daily or hourly? ')
    year = input('Year: ')
    gps_day = input('GPS day: ')
    ftp.cwd(f'{observation_type.lower()}/{year}/{gps_day}')
    igs_station = input('IGS station')

    if observation_type.lower() == 'hourly':
        try:
            os.makedirs(f'Documents/rinex/{year}/{gps_day}/{igs_station}/{observation_type}/')
        except:
            print(f'Folder: Documents/rinex/{year}/{gps_day}/{igs_station}/{observation_type}/ already exists')
        
        for h in hours:
            ftp.cwd(h)
            list_files = ftp.nlst()
            for file in list_files:
                if file.startswith(igs_station) and file.endswith(f'{year[-2:]}{rinex_type}.Z'):
                    print(file)
                    cert_file = file
                    break

            ftp.retrbinary(
                f'RETR {cert_file}',
                open(f'Documents/rinex/{year}/{gps_day}/{igs_station}/{observation_type}/{cert_file}', 'wb').write
            )
            
    
    elif observation_type.lower() == 'daily':
        try:
            os.makedirs(f'Documents/rinex/{year}/{gps_day}/{igs_station}/{observation_type}/')
        except:
            print(f'Folder: Documents/rinex/{year}/{gps_day}/{igs_station}/{observation_type}/ already exists')
        
        ftp.cwd(f'{year[-2:]}{rinex_type}')
        list_files = ftp.nlst()
        for file in list_files:
#             import ipdb; ipdb.set_trace()
            if file.startswith(igs_station) and file.endswith(f'{year[-2:]}{rinex_type}.Z'):
                print(file)
                cert_file = file
                break
        ftp.retrbinary(
            f'RETR {cert_file}',
            open(f'Documents/rinex/{year}/{gps_day}/{igs_station}/{observation_type}/{cert_file}', 'wb').write
        )


In [3]:
def sp3(ftp):
    gps_week = input('GPS week: ')
    gps_day = input('Day on gps week (0-6): ')

    ftp.cwd(f'/gnss/products/{gps_week}')
    product = input('Product (igs, igr, igu): ')
    if product == 'igu':
        hour = input('Two digit hour: ')
    
    search_filename = f'{product}{gps_week}{gps_day}.sp3.Z' if product != 'igu' else f'{product}{gps_week}{gps_day}_{hour}.sp3.Z'
    
    try:
        os.makedirs(f'Documents/sp3/{gps_week}/{gps_day}/')
    except:
        print(f'Folder: Documents/sp3/{gps_week}/{gps_day}/ already exists')
    
    list_files = ftp.nlst()
    for file in list_files:
        if file.lower() == search_filename.lower():
            print(file)
            cert_file = file
            break
    
    ftp.retrbinary(
        f'RETR {cert_file}',
        open(f'Documents/sp3/{gps_week}/{gps_day}/{cert_file}', 'wb').write
    )

In [5]:
ftp = FTP('cddis.nasa.gov')
ftp.login()

file_type = input('File type (SP3 or Rinex): ')

if file_type.lower() == 'rinex':
    rinex(ftp)
elif file_type.lower() == 'sp3':
    sp3(ftp)
else:
    print('Invalid file type')

File type (SP3 or Rinex):  sp3
GPS week:  2107
Day on gps week (0-6):  5
Product (igs, igr, igu):  igs


igs21075.sp3.Z
